# 1. Run Single Cap Tag Report

Run a cap tag report (same output as in Excel).
    
**Inputs** 
1. PR number and revision in form: PR_rev, i.e. '1-IWH6ME_2'
2. Directory to write cap tag report .csv file to, i.e. 'C:\\Users\\wb5888\\Documents\\Cap Tags'

**Outputs**
1. On-screen cap tag report
2. Report exported as .csv file with name CP_PR_rev.csv, i.e. '1-IWH6ME_2.csv' to directory provided above

### Input PR_rev and directory

In [1]:
import os
import numpy as np
import pandas as pd
import CapTags as ct

pr_rev = '1-IZR0VZ_1'

write_dir = 'C:\\Users\\wb5888\\Documents\\Cap Tags'

### View Report

View report that will be exported.

In [2]:
report = ct.get_report(pr_rev)
report

,CustomerName,CustomerID,LDC_Account,AccountID,UIDaccount,MarketCode,ContractID,Revision,StartTime,StopTime,TagType,Tag,SourceType,TimeStamp
0,OLMSTED FALLS BOARD OF EDUCATION,1-5GRZF1,08006052961540024316,PJM_ATSICE_08006052961540024316,1512969,PJM,1-IZR0VZ,1,2018-06-01,2019-05-31 23:59:59,CAPACITY_TAG_OVRD,1.1434,H,2019-05-30 14:56:01
1,OLMSTED FALLS BOARD OF EDUCATION,1-5GRZF1,08006052961540024316,PJM_ATSICE_08006052961540024316,1512969,PJM,1-IZR0VZ,1,2019-06-01,2020-05-31 23:59:59,CAPACITY_TAG_OVRD,1.1434,H,2019-05-30 14:56:58
2,OLMSTED FALLS BOARD OF EDUCATION,1-5GRZF1,08006052961540024316,PJM_ATSICE_08006052961540024316,1512969,PJM,1-IZR0VZ,1,2020-06-01,2021-05-31 23:59:59,CAPACITY_TAG_OVRD,1.1434,H,2019-05-30 14:57:38
3,OLMSTED FALLS BOARD OF EDUCATION,1-5GRZF1,08006052961540024316,PJM_ATSICE_08006052961540024316,1512969,PJM,1-IZR0VZ,1,2021-06-01,2030-05-31 23:59:59,CAPACITY_TAG_OVRD,1.1434,H,2019-05-30 14:58:47
4,OLMSTED FALLS BOARD OF EDUCATION,1-5GRZF1,08037524841020037191,PJM_ATSICE_08037524841020037191,1512965,PJM,1-IZR0VZ,1,2018-06-01,2019-05-31 23:59:59,CAPACITY_TAG_OVRD,0.1542,H,2019-05-30 14:56:01
5,OLMSTED FALLS BOARD OF EDUCATION,1-5GRZF1,08037524841020037191,PJM_ATSICE_08037524841020037191,1512965,PJM,1-IZR0VZ,1,2019-06-01,2020-05-31 23:59:59,CAPACITY_TAG_OVRD,0.1542,H,2019-05-30 14:56:58
6,OLMSTED FALLS BOARD OF EDUCATION,1-5GRZF1,08037524841020037191,PJM_ATSICE_08037524841020037191,1512965,PJM,1-IZR0VZ,1,2020-06-01,2021-05-31 23:59:59,CAPACITY_TAG_OVRD,0.1542,H,2019-05-30 14:57:38
7,OLMSTED FALLS BOARD OF EDUCATION,1-5GRZF1,08037524841020037191,PJM_ATSICE_08037524841020037191,1512965,PJM,1-IZR0VZ,1,2021-06-01,2030-05-31 23:59:59,CAPACITY_TAG_OVRD,0.1542,H,2019-05-30 14:58:47
8,OLMSTED FALLS BOARD OF EDUCATION,1-5GRZF1,08037524841500000473,PJM_ATSICE_08037524841500000473,1512966,PJM,1-IZR0VZ,1,2018-06-01,2019-05-31 23:59:59,CAPACITY_TAG_OVRD,39.5730,H,2019-05-30 14:56:01
9,OLMSTED FALLS BOARD OF EDUCATION,1-5GRZF1,08037524841500000473,PJM_ATSICE_08037524841500000473,1512966,PJM,1-IZR0VZ,1,2019-06-01,2020-05-31 23:59:59,CAPACITY_TAG_OVRD,39.5730,H,2019-05-30 14:56:58


### Check Dates

Ensure tags for all meters cover PR start and end dates.

In [3]:
cap_errors = ct.checkPRdates(report, pr_rev)

date error: ['08004602710000229517', '08004602710000274618', '08004602710001438421', '08004631740001564897', '08004631745000145162', '08004632350001141349', '08004637150001556956', '08004637155000078104', '08004637410000536667', '08004639030000538320', '08004639030000538321', '08004639030000538323', '08004639035000017292', '08004664700001326964', '08004664700001341417', '08004664700001488309', '08004669862090009344', '08004669862090009359', '08004669865000363817', '08004730221050097695', '08004730221560001841', '08004730342440011457', '08004730342440011495', '08004730345000345685', '08004738802750025526', '08004738802750025530', '08004738872010005894', '08004738872030021380', '08004738872030090444', '08004738872270093088', '08004738872410092154', '08004738875000038025', '08004738875000090479', '08004738875000266305', '08004764991850003689', '08004765221810053879', '08004765381900075420', '08004766690001311539', '08004766690001356770', '08004766695000012486', '08004767060000805788', '08

In [15]:
curr = [start.year >= 2018 for start in report.StartTime]
cap = [tag.split('_')[0] == 'CAPACITY' for tag in report.TagType]
cap_summary = report.loc[(curr and cap),['AccountID', 'StartTime', 'StopTime']]
cap_summary.to_csv('PJM_Ohio_Cap_Summary.csv', header = True, index = False)

In [16]:
curr = [start.year >= 2019 for start in report.StartTime]
trans = [tag.split('_')[0] == 'TRANSMISSION' for tag in report.TagType]
trans_summary = report.loc[(curr and trans),['AccountID', 'StartTime', 'StopTime']]
trans_summary.to_csv('PJM_Ohio_Trans_Summary.csv', header = True, index = False)

### Export Report to *write_dir*

If everything looks good report can be exported to the directory specified above.

In [33]:
ct.export_report(pr_rev, write_dir)

saved file as CP_1-IXMUDB_1.csv


# 2. Batch Reports

Batch through PR_rev's and export separate .csv files for each.

**Inputs**
1. List of PR_rev's, i.e. 

```python
['1-IZEA7Q_1', '1-IZ6GRC_1', ...]
```
    
2. Directory to write .csv's to

**Output**
1. Cap report .csv file for each PR_rev given written to given directory

### Give directory and PR_rev list

In [ ]:
write_dir = 'C:\\Users\\wb5888\\Documents\\Cap Tags'

pr_rev_list = ['1-IZEA7Q_1', '1-IZ6GRC_1']

### Iterate through above code

In [ ]:
ct.batch_reports(pr_rev_list, write_dir)

In [ ]:
os.chdir('C:\\Users\\wb5888\\Documents\\Python_Code\\CapReport')

accts = np.unique(report.AccountID)
acct = accts[0]
IDRquery = "select  *  from lschannelcutheader@tppe where channel IN  ( '3','1') and recorder like '" + acct + "%'" 
IDRoutput = ct.OracleAPI(IDRquery)

IDRdf = pd.DataFrame.from_records(IDRoutput)
IDRdf.to_csv('IDRdata_query.csv', header = False, index = False)
IDRdf.head()

### Cap tags: 6/1/xxxx - 5/31/xxxx+1
### Trans tags: 1/1/xxxx - 12/31/xxxx